<a href="https://colab.research.google.com/github/sofia-sunny/Atomic_Neural-Network/blob/main/ANI_2xt_for_tautomers_Energies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Finding the Lowest energy Tautomer(s)**
Here we show the use of [**Auto3D package**](https://pubs.acs.org/doi/abs/10.1021/acs.jcim.2c00817) with the **ANI2xt neural network potential** to generate low energy tautomers of 140  organic molecules in a short time (**less than 3 minutes on T4 GPU**). Auto3D provides a streamlined pipeline for enumerating tautomers, generating 3D conformers, and identifying low-energy tautomeric forms for each molecule.

The dataset used in this tutorial is based on the [**Nicklaus Tautomer Database**](https://pubs.acs.org/doi/abs/10.1021/acs.jcim.9b01156), which includes SMILES and experimental data for 2819 molecules, each with multiple tautomeric forms. We selected only molecules in the gas phase, as Auto3D performs geometry optimization in gas-phase conditions.


In [1]:
import sys,subprocess

In_Colab = 'google.colab' in sys.modules

if In_Colab:
  print("Running in Google Colab")

  from IPython.utils import io

  print("installing required packages...")

  with io.capture_output():
      subprocess.run(
          ["pip", "install", 'auto3d', 'torchani','rdkit','mols2grid']
      )

  print("Installation completed")

Running in Google Colab
installing required packages...
Installation completed


In [2]:
from rdkit import Chem
from Auto3D.auto3D import options
from Auto3D.tautomer import get_stable_tautomers
from rdkit.Chem import PandasTools
import mols2grid
import pandas as pd

First get the the molecules data (Nicklaus Tautomer Database):

In [3]:
all_tauts = pd.read_csv('https://raw.githubusercontent.com/sofia-sunny/Neural-Network/refs/heads/main/data/all_tautomers.csv')

In [4]:
all_tauts.shape

(2819, 147)

So original dataset has 2819 molecules. Each tautomeric entry has been annotated with experimental conditions, bibliographic details, structural identifiers, and chemical information (e.g., SMILES, molecular weight)

In [5]:
all_tauts.head()

,Ref,Size,Solvent,Solvent_Proportion,Solvent_Mixture,Temperature,pH,Experimental_Method,Entry_ID1,Type_1,...,Filename_5,Publication_DOI_5,Publication_ID_5,Authors_5,Affiliation_5,Title_5,Section_5,Page_Number(s)_5,Notes_5,Cmpd_Number_5
0,1,2,Gas phase,nul,no,377.15-417.15,nul,1H NMR spectra,"Prog. NMR. Spect., 2006, 49, 169-206-E-Compoun...",Diketo,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,2,Gas phase,nul,no,nul,nul,nul,"Prog. NMR. Spect., 2006, 49, 169-206-E-Compoun...",NH,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,2,HMPT,nul,no,nul,nul,13C NMR spectra,"Prog. NMR. Spect., 2006, 49, 169-206-E-Compoun...",NH,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,2,DMSO-d6,nul,no,nul,nul,1H NMR spectra,"Prog. NMR. Spect., 2006, 49, 169-206-E-Compoun...",NH,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,3,Carbon tetrachloride,nul,no,nul,nul,1H NMR spectra,"Prog. NMR. Spect., 2006, 49, 169-206-E-Compoun...",Keto-enol,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


As we are only interested in molecules in gas phase:

In [6]:
tauts_gas_state = all_tauts.query('Solvent == "Gas phase"')
tauts_gas_state.shape

(145, 147)

Out of the original 2819 molecules, only 145 were studied in the gas phase. Although each molecule includes between 2 to 5 tautomeric SMILES, we extract only the first tautomer each entry (SMILES_1) for analysis with Auto3D

In [7]:
for col in tauts_gas_state.columns:
  if col.startswith('SMILES'):
    print(col)

SMILES_1
SMILES_2
SMILES_3
SMILES_4
SMILES_5


In [8]:
df = tauts_gas_state[['SMILES_1']].reset_index()
df.head()

,index,SMILES_1
0,0,O=C(C)CC(C)=O
1,1,N1=CC=N[NH]1
2,113,O=S1(CC(C2=CC=CC=C2)=CC=C1)=O
3,590,CC(/C=C(O)/C)=S
4,591,O=C(C)C1=C(S)CCCC1


We keep the original index  for each molecule by renaming the index column to original_ID, allowing easy reference back to the original entry in the Nicklaus Tautomer Database.

In [9]:
df = df.rename(columns={'index': 'original_ID'})[['SMILES_1','original_ID']]

In [10]:
df.head()

,SMILES_1,original_ID
0,O=C(C)CC(C)=O,0
1,N1=CC=N[NH]1,1
2,O=S1(CC(C2=CC=CC=C2)=CC=C1)=O,113
3,CC(/C=C(O)/C)=S,590
4,O=C(C)C1=C(S)CCCC1,591


Since the **ANI2xt** model in Auto3D only supports molecules containing the elements H, C, N, O, F, Cl, and S, we filter out any SMILES that include unsupported elements. The function **is_allowed** checks each atom in the molecule and returns False if any element falls outside the allowed set

In [11]:
allowed_elements = {'H', 'C', 'N', 'O', 'F', 'Cl', 'S'}

def is_allowed(smiles):
    mol = Chem.MolFromSmiles(smiles)
    for atom in mol.GetAtoms():
        if atom.GetSymbol() not in allowed_elements:
            return False
    return True

df_final = df[df['SMILES_1'].apply(is_allowed)].reset_index(drop=True)
df_final.shape

(140, 2)


After filtering, we have 140 molecules  that are compatible with ANI2xt.

We save these molecules in a **.smi** file to use as input for Auto3D.


In [12]:
input_file = 'molecules.smi'
with open(input_file, 'w') as f:
    for index, row in df_final.iterrows():
        f.write(f"{row['SMILES_1']}\t{row['original_ID']}\n")

### **Running the Auto3D**
The **options function** in Auto3D is used to set the parameters for controlling the 3D structure generation and optimization workflow.

**The steps to find the  low-energy tautomer(s) by Auto3D:**

1. Enumerating reasonable tautomers for each input SMILES (enumerate_tautomer=True)

2. Getting top **k conformers** for each tautomer by grouping all conformers of all tautomers of the same SMILES together
3. selecting the **top tauto_k conformers** as the final stable tautomer 3D structures for the input SMILE
The tautomers are ranked based on their conformer energies and therfore the 1st step is to generate conformers for all tautomers.

The conformer generation step sets **max_confs=10** and **patience=200**. Because here we care more about the relative stabilities of tautomers, 10 conformers from each tautomer would be a good representation whereas maintaining high efficiency.

Other parameters in options function:

**enumerate_tautomer:** If True, Auto3D will generate possible tautomeric forms of the input molecules.

**tauto_engine:** Specifies the engine used for tautomer enumeration (e.g., "**rdkit**").

**optimizing_engine:** Defines the neural network potential used for geometry optimization (**ANI2xt** here).


In [13]:
args = options(input_file, k=1, enumerate_tautomer=True, tauto_engine="rdkit",
                optimizing_engine="ANI2xt",
                max_confs=5, patience=200, use_gpu=True)


**get_stable_tautomer** function directly accepts the arguments from the options function and returns the **tatuo_k** tautomers with lowest energies. (this takes less than 3 minutes on T4 GPU)


In [14]:
tautomer_out = get_stable_tautomers(args, tauto_k=3)

INFO:auto3d:
         _              _             _____   ____  
        / \     _   _  | |_    ___   |___ /  |  _ \ 
       / _ \   | | | | | __|  / _ \    |_ \  | | | |
      / ___ \  | |_| | | |_  | (_) |  ___) | | |_| |
     /_/   \_\  \__,_|  \__|  \___/  |____/  |____/  2.3.1
              // Generating low-energy 3D structures                                      
    
INFO:auto3d:================================================================================
INFO:auto3d:                               INPUT PARAMETERS
INFO:auto3d:================================================================================
INFO:auto3d:path: molecules.smi
INFO:auto3d:k: 1
INFO:auto3d:window: False
INFO:auto3d:verbose: False
INFO:auto3d:job_name: 20250706-235025-965488
INFO:auto3d:enumerate_tautomer: True
INFO:auto3d:tauto_engine: rdkit
INFO:auto3d:pKaNorm: True
INFO:auto3d:isomer_engine: rdkit
INFO:auto3d:enumerate_isomer: True
INFO:auto3d:mode_oe: classic
INFO:auto3d:mpi_np: 4
INFO:auto3d:m

Checking input file...


INFO:auto3d:Checking input file...


	There are 140 SMILES in the input file molecules.smi. 
	All SMILES and IDs are valid.


INFO:auto3d:	There are 140 SMILES in the input file molecules.smi. 
	All SMILES and IDs are valid.


...

INFO:auto3d:Program running time: 3 minute(s)


Output path: /content/molecules_20250706-235025-965488/molecules_out.sdf


INFO:auto3d:Output path: /content/molecules_20250706-235025-965488/molecules_out.sdf



Begin to select stable tautomers based on their conformer energies...
Done.
The stable tautomers are stored in: /content/molecules_20250706-235025-965488/molecules_out_top_tautomers.sdf


/usr/local/lib/python3.11/dist-packages/Auto3D/tautomer.py:31: FutureWarning: When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.
  group = groups.get_group(group_name)


We can now convert the tautomer_out results into a DataFrame for easier analysis and visualization

In [15]:
output_df = PandasTools.LoadSDF(tautomer_out)

In [16]:
output_df.head()

,ID,E_tot,fmax,Converged,E_rel(kcal/mol),E_tautomer_relative(kcal/mol),ROMol
0,0,-345.6534345398188,0.0028853153344243765,True,0.0,0.0,<rdkit.Chem.rdchem.Mol object at 0x790bbe3460a0>
1,0,-345.6281300807834,0.0029361837077885866,True,0.0,15.878787763329456,<rdkit.Chem.rdchem.Mol object at 0x790bbe345f50>
2,0,-345.61376201608186,0.002890688134357333,True,0.0,24.89488447763142,<rdkit.Chem.rdchem.Mol object at 0x790bbe396180>
3,1,-242.11985834420918,0.0025867256335914135,True,0.0,0.0,<rdkit.Chem.rdchem.Mol object at 0x790bbe3958c0>
4,1,-242.1124620783448,0.0029316258151084185,True,0.0,4.641226897481829,<rdkit.Chem.rdchem.Mol object at 0x790bbe3962d0>


**mols2grid library** can create a grid of molecule images, making it easier to explore chemical structures along with associated data.

The **mol_col**='ROMol' argument specifies which column in the DataFrame contains RDKit molecule objects to render.

The **subset parameter** selects the columns to display alongside each molecule—in this case, the **molecule's ID** and its **relative tautomer energy** in kcal/mol.

The **transform parameter** is used to format the energy values using the custom function (**two_decimal**), which lrounds the number to two decimal places for cleaner visualization.

In [17]:
two_decimal = lambda x: round(float(x), 2)

In [ ]:
mols2grid.display(output_df, mol_col='ROMol', subset=['ID',"E_tautomer_relative(kcal/mol)"], transform = {"E_tautomer_relative(kcal/mol)": two_decimal})

 The checkboxes above each molecule allow for interactive selection, which can be useful for further filtering or exporting selected compounds

**Acknowledgment**

Thanks to [Pat Walters](https://github.com/PatWalters) for providing the  original tutorial on using Auto3D